# LLM 모델 Fine tuning

## LLM은 Foundation(기반) 모델
- **광범위한 데이터로 학습한 Pretrained 모델**
    - LLM은 대규모의 텍스트 데이터로 사전 학습되며, 다양한 주제와 언어에 대한 지식을 포함하고 있다.
- **범용성**
    - LLM은 **특정 작업에 최적화되지 않았지만**, 다양한 자연어 처리 작업에 적용될 수 있는 기초 역량을 제공한다.
    - 일반적인 NLP task인 텍스트생성, 문서요약, 번역, 질의응답에서 좋은 성능을 보인다.

## Fine tuning의 필요성
Fine-tuning을 통해 범용적인 Foundation 모델을 **특정 작업, 도메인, 사용자 요구에 맞게 최적화**하여 모델의 더 좋은 성능을 확보할 수있다.
-  **특정 작업에 최적화**  
   - Foundation 모델은 범용적인 언어 이해와 생성 능력을 가지지만, 특정 도메인(업무)이나 작업에 맞춰져 있지 않다.  
   - Fine-tuning은 특정 목적에 맞게 모델을 조정하여 성능을 극대화 할 수 있다.
- **도메인 특화 지식 추가**  
   - LLM 모델이 학습한 사전 학습 데이터는 일반적인 정보를 포함하지만, 전문 도메인 지식이 부족할 수 있다. 
   - 회사에서 LLM 모델을 사용할 경우 범용 모델은 그 회사의 내부 데이터를 학습하지 못했기 때문에, 그 내용에 대한 지식이 없다. 
   - Fine-tuning을 통해 전문 도메인 데이터나 회사내 데이터로 추가 학습시켜 전문성을 확보할 수 있다. 
-  **사용자 맞춤 응답 제공**  
   - Fine-tuning은 특정 조직이나 서비스의 언어 스타일과 고객 요구에 맞게 응답을 최적화할 수 있다.
- **안전성과 윤리 강화**  
   - 모델이 민감한 주제나 잘못된 정보에 대해 잘못된 답변을 하지 않도록 추가 학습을 통해 제어 가능하다.

## LLM Fine tuning으로 발생할 수 있는 문제
- **고성능 하드웨어 필요**
    - LLM은 최소 수십억개에서 조단위의 매개변수를 포함가지는 모델이다. 그래서 Fine-tuning하려면  이전의 딥러닝 모델에 비해 많은  GPU를 가지는 고성능 하드웨어가 필요하다.
- **오랜 학습시간**
    - Fine-tuning은 데이터 전처리, 학습, 평가에 많은 시간이 소요된다.
- **큰 비용**
    - 위와 같은 이유로 Fine-tuning은 많은 비용이 소요된다.
- **도메인 한정성**
    - 특정 도메인에 맞춘 모델이 다른 작업에서 성능이 떨어질 수 있다.
    - 또한 기존  Foundation 모델의 범용적인 지식을 일부 덮어쓰거나 왜곡할 가능성이 있다.

# Fine tuning 방법
- 인프라(하드웨어, 오랜 학습시간)과 연관되어 fine tuning 방법을 본다.
1. 모델의 크기 줄이기
2. 학습양을 줄일 수 있는 다양한 학습방법 개발


# Quantization 개요
- 모델의 크기를 줄이기 위해 파라미터의 정보를 최대한 유지하면서 타입을 줄이는 것을 quantization(양자화) 라고 한다.
  
## Data Type 관점에서 양자화
- 딥러닝 모델들은 파라미터를 부동소수점(floating point) 타입으로 저장한다 그리고 일반적으로 32비트를 사용한다. (float32: 단정밀도)
    - 부동 소수점 타입은 값을 지수와 가수로 나눠 저장한다.
    - 지수: 표현할 수 있는 숫자의 범위를 결정한다.
    - 가수: 정밀도를 결정한다. 정밀도는 연속형 값의 촘촘함/세밀함라고 할 수있다.
    - 1234를 부동소수점으로 표현: 1.234 X 10^3
        - 지수: 3 (10^3), 가수: 1.234
- fp32
    - 지수: 8 bit, 가수: 23 bit, 부호: 1 bit
- fp16
    - 지수: 5 bit, 가수: 10 bit, 부호: 1 bit
- bf16 (Brain Floating Point)
    - fp16이 fp32에 비해 표현할 수있는 수의 범위가 좁아서 타입 변환시 정보가 많이 사라지는 문제가 있다. 
    - 그래서 이를 보완하기 위해 google brain에서 제안한 타입으로 지수부의 크기를 fp32와 동일하게 8bit를 사용한다. 가수부의 크기가 줄어 들어 정밀도를 낮아지지만 더 넓은 범위의 값을 표현하도록 하였다.
    - 지수: 8bit, 가수: 7bit, 부호: 1 bit

  ![floating_type](figures/quantize1.png)

  - fp32 를 fp16으로 줄이면 1/2 로 크기가 줄어든다. 그러나 fp16이 표현할 수 있는 값의 범위가 줄어 들면서 정보의 손실이 크게 발생할 수있다.
  - bf16은 fp32에 비해 1/2로 크기가 줄어들지만, fp32와 비슷한 범위의 값을 표현할 수 있다. 단 정밀도가 낮아 진다.
  - 비트수를 더 줄이면(8bit) 정수기반 표현을 하게 된다. 

## Quantization 방법
- **양자화는 모델 파라미터의 정보를 최대한 유지하면서 타입을 줄이는 것을 말한다.**
    - 일정 부분 정보손실을 감수하면서 최대한 크기를 줄이는 것이 목적이다.
    - 파라미터의 정보손실로 성능은 떨어질 수있지만 모델의 크기를 줄여서 메모리 사용량을 줄이고 속도를 높일 수 있다.
    - 이 개념은 우리가 시간을 물어볼 때, "10시 28분 22초" 대신 "10시 28분" 또는 "10시 반" 이라고 말해도 충분히 이해할 수 있는 상황과 같다. 약간의 정보 손실은 있지만, 질문에 대한 답으로는 충분하다는 점에서 유사하다.

### Int 8 Quantization
- INT8 양자화는 딥러닝 **모델의 파라미터와 활성화 값(뉴런의 출력값)을** 32비트 부동소수점(FP32)에서 **8비트 정수(INT8)로 변환**하는 기법이다.
- 이를 통해 모델의 메모리 사용량을 대략 75% 정도 감소시킬 수있다. 또한 추론 속도가 하드웨어와 모델에 따라 2 ~ 4배 정도 향상된다.

#### **Absmax Quantization**
fp32를 int8 로 양자화하는 가장 단순한 기법. 

![figure](figures/quantize2.png)

**주요 단계:**
1. **최대 절대값 찾기**
   - 주어진 텐서(fp32)에서 **절대값이 가장 큰 값**을 찾는다.

2. **스케일 팩터 계산**
   - **스케일 팩터(scale factor)는** 값의 scale을 변경하기 위해 값에 곱해주는 값을 말한다. 
   - 변경하려는 데이터타입(int8)의 최대값을 1에서 구한 절대값으로 나눠 스케일 팩터를 계산한다. 
   - `int8`의 경우 최대값은 127이다.

3. **양자화**
   - 각 원소에 스케일 팩터를 곱하고 반올림하여 정수형으로 변환한다.

4. **예**

    - 주어진 텐서: `[1.2, -0.5, -4.3, 1.2, -3.1, 0.8, 2.4, 5.4]`
    - 최대 절대값: `5.4`
    - 스케일 팩터: `127 / 5.4 ≈ 23.5`
    - 양자화된 값: `[28, -12, -101, 28, -73, 19, 56, 127]`

5. **장점:**
    - 연산량이 적다.
    - 데이터의 분포가 0을 중심으로 대칭적일 때 효과적이다.

6. **단점:**
    - 데이터에 **이상치(Outlier)가** 에 취약하다.
        - 스케일 팩터가 왜곡되어 양자화의 정확도가 떨어질 수 있다.
        - 공간의 낭비가 발생한다.
  
  ![figure](figures/quantize3.png)

7. **absmax 양자화 단점 보완**
    - 전체를 기준으로 변환을 수행하는 것이 아니라 K개씩 데이터를 묶어서 block 단위로 양자화 한다.
    - 대상 텐서를 K개 씩 묶은 뒤 그 안에서 절대 최댓값을 구해서 변환을 수행한다. 
    - 이렇게 하면 이상치와 같이 묶인 block의 tensor 들만 이상치에 영향을 받는다.
 

## 4-bit NormalFloat Quantization
- QLoRA 튜닝 방법에서 제안한 양자화 방식으로, 모델의 메모리 사용량을 대폭 줄이면서도 성능을 유지하는 기법이다.
- NF4는 일반적으로 사용되는 32비트 부동소수점(FP32) 대신 **4비트만**을 사용하여 데이터를 표현한다.
- 이는 분위수 양자화(Quantile Quantization)를 기반으로 하며, 각 양자화 구간에 동일한 수의 값이 할당되도록 설계되었다.
    - 분위수 양자화(Quantile Quantization)는 양자화시 분위수로 구간을 나누고 각 구간에 동일한 수의 값이 포함하도록 하는 방식이다.

### 양자화 과정
- 모델의 파라미터를 **평균이 0이고 표준편차가 $\sigma$인 정규 분포로 가정**한다.
    - 딥러닝 모델의 파라미터들이 표준 정규 분포를 따르는 경우가 많다는 사실에서 시작한다.
  
1. 신경망 가중치는 **정규분포**를 따르므로, 이를 기반으로 **양자화 구간**을 설정한다.
2. **표준 정규분포**에서 **$2^k$개의 구간**을 만든다.
    - 표준 정규분포는 평균 근처에 더 많은 데이터가 분포하므로 평균(0) 주변에는 **좁은 구간**이, 양 끝 부분에는 **넓은 구간**이 형성된다.
3. 양자화 구간의 범위를 [-1 , 1] 사이로 정규화한다.

![figure](figures/quantize4.png)

4. 대상 tensor들도 **absmax** 값으로 나눠 [-1 , 1] 범위로 정규화(scaling)한다.
5. 4에서 정규화 한 값을 양자화된 구간의 대표값으로 대체한다.
    1. scaling 된 값이 0.044. 양자화 구간 0.04 ~ 0.05. 그럼 scaling 된 값은 이 구간에 속한다.
    2. 그럼 0.044를 구간의 대표값 (ex 중앙값) 으로 대체한다.
        - 양자화된 값 = (0.04 + 0.05) / 2 = 0.045

In [ ]:
%pip install bitsandbytes transformers accelerate tqdm ipywidgets -U

### Model Load

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_id = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Print model size
print(f"Model size: {model.get_memory_footprint():,} bytes")

Model size: 510,342,192 bytes


## 양자화 설정 후 load

In [3]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

print(f"model_4bit Model size: {model_4bit.get_memory_footprint():,} bytes")

PackageNotFoundError: No package metadata was found for bitsandbytes